# Introduction 

This note evaluate phi3-mini-4k-instruct with llama.cpp

# Env and tools prepare

In [1]:
!cp -r /kaggle/input/llama-cpp-binary-compiled-with-gpu/llama.cpp /kaggle/working/

In [2]:
!chmod +x -R llama.cpp
%cd llama.cpp
!cp -r /usr/local/cuda-12.3/targets /usr/local/nvidia/ 

/kaggle/working/llama.cpp


# Run perplexity

Perplexity is the main evaluation metric for large language models (LLMs). It measures how well the model predicts a given sequence of tokens.
Formally, the perplexity of LLMs is the exponentiated average negative log-likelihood,  which means **perplexity smaller is better** 

Perplexity benchmark shows how accurate the model is 

In [3]:

!/kaggle/working/llama.cpp/llama-perplexity -m /kaggle/input/phi3-mini-4k-instruct-q4_k_m/gguf/q4_k_m/1/Phi3-mini-4k-instruct-Q4.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4.log 2>&1


In [4]:
!tail -6 ppl_Q4.log


Final estimate: PPL = 7.1613 +/- 0.07725

llama_perf_context_print:        load time =   14828.09 ms
llama_perf_context_print: prompt eval time =  371183.33 ms / 131072 tokens (    2.83 ms per token,   353.12 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  391160.00 ms / 131073 tokens


# Benchmarking the Inference Throughput and Memory Consumption 

Llama-bench shows performance and resource consumption. 
There are three types of test
- Prompt processing (pp): processing a prompt in batches (-p)
- Text generation (tg): generating a sequence of tokens (-n)
- Prompt processing + text generation (pg): processing a prompt followed by generating a sequence of tokens (-pg)

In the following test, we run text generation and prompt processing


In [5]:
!/kaggle/working/llama.cpp/llama-bench -m /kaggle/input/phi3-mini-4k-instruct-q4_k_m/gguf/q4_k_m/1/Phi3-mini-4k-instruct-Q4.gguf -mg 1 -ngl 10,20,30,35 -n 128,256,512 

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla P100-PCIE-16GB, compute capability 6.0, VMM: yes
| model                          |       size |     params | backend    | ngl |   main_gpu |          test |                  t/s |
| ------------------------------ | ---------: | ---------: | ---------- | --: | ---------: | ------------: | -------------------: |
| phi3 3B Q4_K - Medium          |   2.23 GiB |     3.82 B | CUDA       |  10 |          1 |         pp512 |        462.15 ± 0.53 |
| phi3 3B Q4_K - Medium          |   2.23 GiB |     3.82 B | CUDA       |  10 |          1 |         tg128 |          8.03 ± 0.13 |
| phi3 3B Q4_K - Medium          |   2.23 GiB |     3.82 B | CUDA       |  10 |          1 |         tg256 |          7.87 ± 0.05 |
| phi3 3B Q4_K - Medium          |   2.23 GiB |     3.82 B | CUDA       |  10 |          1 |         tg512 |          7.54 ± 0.03 |
| phi3 3B Q4_K - 